In [1]:
!pip install scikit-learn==0.21.3

     |████████████████████████████████| 6.7 MB 9.3 MB/s eta 0:00:01
ERROR: tpot 0.11.1 has requirement scikit-learn>=0.22.0, but you'll have scikit-learn 0.21.3 which is incompatible.
ERROR: kmeans-smote 0.1.2 has requirement imbalanced-learn<0.5,>=0.4.0, but you'll have imbalanced-learn 0.6.2 which is incompatible.
ERROR: kmeans-smote 0.1.2 has requirement numpy<1.16,>=1.13, but you'll have numpy 1.18.2 which is incompatible.
ERROR: kmeans-smote 0.1.2 has requirement scikit-learn<0.21,>=0.19.0, but you'll have scikit-learn 0.21.3 which is incompatible.
ERROR: imbalanced-learn 0.6.2 has requirement scikit-learn>=0.22, but you'll have scikit-learn 0.21.3 which is incompatible.
ERROR: cesium 0.9.12 has requirement scikit-learn>=0.22.1, but you'll have scikit-learn 0.21.3 which is incompatible.
ERROR: allennlp 0.9.0 has requirement spacy<2.2,>=2.1.0, but you'll have spacy 2.2.3 which is incompatible.
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2

In [2]:
# Import the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.metrics import mean_squared_error

warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('max_column', 1000)

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/covid-19/Train_maskedv2.csv
/kaggle/input/covid-19/Test_maskedv2.csv
/kaggle/input/covid-19/variable_descriptions_v2.csv
/kaggle/input/covid-19/samplesubmissionv2.csv


In [4]:
train = pd.read_csv('/kaggle/input/covid-19/Train_maskedv2.csv')
test = pd.read_csv('/kaggle/input/covid-19/Test_maskedv2.csv')
sample = pd.read_csv('/kaggle/input/covid-19/samplesubmissionv2.csv')
description = pd.read_csv('/kaggle/input/covid-19/variable_descriptions_v2.csv')

In [5]:
pd.set_option('max_colwidth', 200)
pd.set_option('max_info_rows', 1000)

In [6]:
train.drop(columns=['dw_12', 'dw_13', 'lan_13'], inplace=True)
test.drop(columns=['dw_12', 'dw_13', 'lan_13'], inplace=True)

In [7]:
train_len = len(train)
data=pd.concat([train,test])

In [8]:
data['rich'] = data['car_01']+data['stv_01']+data['psa_01']+data['dw_02']+data['lln_00']
data['poor'] = data['car_00'] +data['stv_00']+data['psa_00']+data['dw_01']+data['lln_01']

In [9]:
data['household_size'] =data['total_individuals'] / data['total_households']

In [10]:
from sklearn.cluster import KMeans
columns=data.drop(["target_pct_vunerable","ward"],1).columns

data_km=data[columns].copy()

data_km["total_households"]/=data_km["total_households"].max()
data_km["total_individuals"]/=data_km["total_individuals"].max()

km=KMeans(15,random_state=2019)
data["cluster"]=km.fit_predict(data_km[columns])

In [11]:
train = data[:train_len]
test = data[train_len:]

In [12]:
_id = test['ward']
test.drop(columns=['target_pct_vunerable','ward'], inplace=True)
train.drop(columns=['ward'], inplace=True)

In [13]:
train.shape, test.shape

((3174, 50), (1102, 49))

In [14]:
train['total_households'] = np.log10(train['total_households'])
test['total_households'] = np.log10(test['total_households'])

train['total_individuals'] = np.log10(train['total_individuals'])
test['total_individuals'] = np.log10(test['total_individuals'])

In [15]:
X = train.drop(columns=['target_pct_vunerable'])
y = train['target_pct_vunerable']

In [16]:
X.columns

Index(['car_00', 'car_01', 'dw_00', 'dw_01', 'dw_02', 'dw_03', 'dw_04',
       'dw_05', 'dw_06', 'dw_07', 'dw_08', 'dw_09', 'dw_10', 'dw_11', 'lan_00',
       'lan_01', 'lan_02', 'lan_03', 'lan_04', 'lan_05', 'lan_06', 'lan_07',
       'lan_08', 'lan_09', 'lan_10', 'lan_11', 'lan_12', 'lan_14', 'lgt_00',
       'lln_00', 'lln_01', 'pg_00', 'pg_01', 'pg_02', 'pg_03', 'pg_04',
       'psa_00', 'psa_01', 'psa_02', 'psa_03', 'psa_04', 'stv_00', 'stv_01',
       'total_households', 'total_individuals', 'rich', 'poor',
       'household_size', 'cluster'],
      dtype='object')

In [17]:
X.isna().sum().any()

False

In [18]:
col = ['car_00', 'car_01', 'dw_00', 'dw_01', 'dw_02', 'dw_03', 'dw_04',
       'lan_08', 'lan_09', 'lan_10', 'lan_11', 'lan_12', 'lan_14', 'lgt_00',
       'dw_05', 'dw_06', 'dw_07', 'dw_08', 'dw_09', 'dw_10', 'dw_11', 'lan_00',
       'lan_01', 'lan_02', 'lan_03', 'lan_04', 'lan_05', 'lan_06', 'lan_07',
       'lln_00', 'lln_01', 'pg_00', 'pg_01', 'pg_02', 'pg_03', 'pg_04',
       'psa_00', 'psa_01', 'psa_02', 'psa_03', 'psa_04', 'stv_00', 'stv_01',
        'rich', 'poor']

In [19]:
X[col] = X[col].round(2)
test[col] = test[col].round(2)

In [20]:
categorical_features_indices = np.where(X.dtypes != np.float)[0]; categorical_features_indices

array([48])

In [ ]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit

testsplit_store=[]
test_store=[]
fold=KFold(n_splits=15, shuffle=True, random_state=123456)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    cat = CatBoostRegressor(n_estimators=10000,eval_metric='RMSE', learning_rate=0.0801032, random_seed= 123456, l2_leaf_reg=4, use_best_model=True)
    cat.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=300,verbose=100, cat_features=categorical_features_indices)
    predict = cat.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,predict)))
    testsplit_store.append(np.sqrt(mean_squared_error(y_test,predict)))
    pred = cat.predict(test)
    test_store.append(pred)

In [ ]:
np.mean(testsplit_store)

In [ ]:
submit_prep = {"ward": _id, 'target_pct_vunerable': np.mean(test_store, 0)}
submission = pd.DataFrame(data = submit_prep)

In [ ]:
submission.head()

In [ ]:
submission.to_csv('covid_19_best.csv', index=False)

In [ ]:
## Check for the feature importance 
fea_imp = pd.DataFrame({'imp':cat.feature_importances_, 'col': X.columns})
fea_imp = fea_imp.sort_values(['imp', 'col'], ascending=[True, False]).iloc[-60:]
_ = fea_imp.plot(kind='barh', x='col', y='imp', figsize=(20, 10))
plt.savefig('catboost_feature_importance.png')